# molPX intro
<pre> 
Guillermo Perez-Hernandez  guille.perez@fu-berlin.de 
</pre>
   
In this notebook we will be using the 1 millisecond trajectory of Bovine Pancreatic Trypsin Inhibitor (BPTI) generated by DE Shaw Research on the Anton Supercomputer and kindly made available by their lab. The original work is 
 
 * Shaw DE, Maragakis P, Lindorff-Larsen K, Piana S, Dror RO, Eastwood MP, Bank JA, Jumper JM, Salmon JK, Shan Y, Wriggers W: Atomic-level characterization of the structural dynamics of proteins. Science 330:341-346 (2010).  doi: 10.1126/science.1187409.
 
The trajectory has been duplicated and shortened to provide a mock-trajectory set and be able to deal with lists of trajectories of different lenghts:

 * `c-alpha_centered.stride.100.xtc`
 * `c-alpha_centered.stride.100.reversed.xtc`
 * `c-alpha_centered.stride.100.halved.xtc`

## Input types and typical usecase
The typical usecase is having molecular dynamics (MD) simulation data in form of trajectory files with extensions like `.xtc, .dcd` etc and the associated molecular topology as a `.pdb` or `.gro` file. 

These files are the most general starting point for any analysis dealing with MD, and ```molpx```'s API has been designed to be able to function without further input:

In [2]:
top = 'notebooks/data/bpti-c-alpha_centered.pdb'
MD_trajfiles = ['notebooks/data/c-alpha_centered.stride.1000.xtc',
               'notebooks/data/c-alpha_centered.stride.1000.reversed.xtc',
                'notebooks/data/c-alpha_centered.stride.1000.halved.xtc'
               ]

dt = 244 #saving interval in the .xtc files, in ns

import molpx
from matplotlib import pyplot as plt
import pyemma
import numpy as np

# This way the user does not have to care where the data are:
top = molpx._molpxdir(top)
MD_trajfiles = [molpx._molpxdir(ff) for ff in MD_trajfiles]

**However**, `molpx` relies heavily on the awesome [`mdtraj`](http://www.mdtraj.org) module for dealing with molecular structures, and so most of `molpx`'s functions accept also `Trajectory`-type objects (native to `mdtraj`) as alternative inputs. 

In [3]:
# Create a memory representation of the trajectories
MD_list = [molpx.generate._md.load(itraj, top=top) for itraj in MD_trajfiles]

The same idea applies to the input of projected trajectories: `molpx` can take the filenames as inputs (`.npy`, `.dat`, `.txt` etc) or deal directly with `numpy.ndarray` objects. 

** These alternative, "from-memory" input modes (`md.Trajectory` and `np.ndarray` objects) avoid forcing the user to read from file everytime an API function is called, saving I/O overhead**

The following cell either reads or generates projected trajectory files for this demonstration. In a real usecase this step (done here using TICA) might not be needed, given that the user might have generated the projected trajectory elsewhere:

In [4]:
# Perform TICA or read from file directly if already .npy-files exist
Y_filenames = [ff.replace('.xtc','.Y.npy') for ff in MD_trajfiles]
try:        
    Y = [np.load(ff) for ff in Y_filenames]
except:
    feat = pyemma.coordinates.featurizer(top)
    pairs = feat.pairs(range(feat.topology.n_atoms)[::2])
    feat.add_distances(pairs)
    src  = pyemma.coordinates.source(MD_trajfiles, features=feat)
    tica = pyemma.coordinates.tica(src, lag=10, dim=3)
    Y = tica.get_output()        
    [np.save(ff, iY) for ff, iY in zip(Y_filenames, Y)]

## Visualize a FES and the molecular structures behind it
Execute the following cell and click either on the FES or on the slidebar. Some input parameters have been comented out for you to try out:
 * different modes of input (disk vs memory) 
 * different projection indices
 * different number of overlaid structures

In [5]:
mpx_wdg_box = molpx.visualize.FES(MD_list, 
                                 #MD_trajfiles, 
                                 top, 
                                 Y_filenames, 
                                 #Y, 
                                 nbins=50, 
                                 #proj_idxs=[1,2],
                                 proj_labels='TIC',
                                 #n_overlays=5,
                                )
mpx_wdg_box

A Jupyter Widget

30-05-18 17:43:04 pyemma.coordinates.clustering.kmeans.KmeansClustering[0] INFO     Cluster centers converged after 6 steps.


A Jupyter Widget

A Jupyter Widget

## Visualize trajectories, FES and molecular structures
The user can sample structures as they occurr in sequence in the actual trajectory. Depending on the size of the dataset, this can be very time consuming, particularly if data is being read from disk. 

In this example, try changing `MD_trajfiles` to `MD_list` and/or changing `Y_filenames` to simply `Y` and see if it helps.

Furthermore, the objects in memory can be strided down to fewer frames **before** being parsed to the method. To stride objects being read from the disk, use the `stride` parameter. 

### Again, we have commented out some parameters that provide more control on the output of `visualize.traj`. Uncomment them and see what happens

In [6]:
mpx_wdg_box = molpx.visualize.traj(MD_trajfiles, 
                                   top, 
                                   Y,
                                   #Y_filenames,                                          
                                   plot_FES = True,                                        
                                   dt = dt*1e-6, tunits='ms',    
                                   #traj_selection = 1,
                                   #sharey_traj=False,
                                   #max_frames=100,
                                   proj_idxs=[0, 1],
                                   panel_height=2,    
                                   #proj_labels='TIC'
                                  )
mpx_wdg_box

A Jupyter Widget

A Jupyter Widget

30-05-18 17:43:10 pyemma.coordinates.clustering.kmeans.KmeansClustering[10] INFO     Algorithm did not reach convergence criterion of 1e-05 in 10 iterations. Consider increasing max_iter.


A Jupyter Widget

A Jupyter Widget

# Intermediate steps: using molpx to generate a regspace sample of the data
See the documentation of `molpx.generate.sample` to find out about all possible options:
```
molpx.generate.sample(MD_trajectories, MD_top, projected_trajectories, atom_selection=None, proj_idxs=[0, 1], n_points=100, n_geom_samples=1, keep_all_samples=False, proj_stride=1, verbose=False, return_data=False)
```

In [7]:
data_sample, geoms = molpx.generate.sample(#MD_list, 
                                           MD_trajfiles, 
                                           top, 
                                           #Y, 
                                           Y_filenames,
                                           n_points=200   ,
                                            n_geom_samples=2,
                                    )
data_sample.shape, geoms


A Jupyter Widget

A Jupyter Widget

30-05-18 17:43:15 pyemma.coordinates.clustering.kmeans.KmeansClustering[19] INFO     Algorithm did not reach convergence criterion of 1e-05 in 10 iterations. Consider increasing max_iter.


A Jupyter Widget

((203, 2),
 <mdtraj.Trajectory with 203 frames, 58 atoms, 58 residues, and unitcells at 0x7f56cb248860>)

## Link the PDF plot with the sampled structures and visually explore the FES 
Click either on the plot or on the widget slidebar: they're connected! 

In [8]:
# Replot the FES
plt.figure(figsize=(7,7))
h, (x,y) = np.histogramdd(np.vstack(Y)[:,:2], bins=50)
plt.contourf(x[:-1], y[:-1], -np.log(h.T), alpha=.50)
# Create the linked widget
linked_ngl_wdg, linked_ax_wdg = molpx.visualize.sample(data_sample, 
                              geoms.superpose(geoms[0]),                             
                              plt.gca(), 
                              clear_lines=True,
                              #plot_path=True
                            )
plt.plot(data_sample[:,0], data_sample[:,1],' ok', zorder=0)
# Show it
linked_ngl_wdg


A Jupyter Widget

/home/mi/gph82/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


A Jupyter Widget

# Paths samples along the different projections (=axis)

In [9]:
paths_dict, idata = molpx.generate.projection_paths(#MD_list, 
                                                    MD_trajfiles, 
                                                    top, 
                                                    Y_filenames,
                                                    #Y, # You can also directly give the data here
                                                    n_points=50,
                                                    proj_idxs=[0,1],
                                                    n_projs=3,
                                                    proj_dim = 3, 
                                                    verbose=False, 
                                        )

A Jupyter Widget

A Jupyter Widget

30-05-18 17:43:20 pyemma.coordinates.clustering.kmeans.KmeansClustering[30] INFO     Algorithm did not reach convergence criterion of 1e-05 in 10 iterations. Consider increasing max_iter.


A Jupyter Widget

A Jupyter Widget

30-05-18 17:43:26 pyemma.coordinates.clustering.kmeans.KmeansClustering[37] INFO     Cluster centers converged after 11 steps.


A Jupyter Widget

## Link the PDF plot with the sampled paths/structures and visually explore the coordinates (separately). 
Click either on the plot or on the widget slidebar: they're connected! You can change the type of path between min_rmsd or min_disp and you can also change the coordinate sampled (0 or 1)


In [10]:
# Choose the coordinate and the tyep of path
coord = 1
#path_type = 'min_rmsd'
path_type = 'min_disp'
igeom = paths_dict[coord][path_type]["geom"]
ipath = paths_dict[coord][path_type]["proj"]

# Choose the proj_idxs for the path and the FES 
# to be shown
proj_idxs = [0,1]

In [11]:
plt.figure(figsize=(7,7))
h, (x,y) = np.histogramdd(np.vstack(Y)[:,proj_idxs], bins=50)
plt.contourf(x[:-1], y[:-1], -np.log(h.T), alpha=.50)

linked_ngl_wdg, linked_ax_wdg = molpx.visualize.sample(ipath[:,proj_idxs], 
                                    igeom.superpose(igeom[0]),                             
                                    plt.gca(), 
                                    clear_lines=True,
                                    n_smooth = 5, 
                                    plot_path=True,                        
                                    #radius=True,
                            )
linked_ngl_wdg

A Jupyter Widget

/home/mi/gph82/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


A Jupyter Widget

# Interaction with ```PyEMMA```
`molpx` is using many methods of the `coordinates` submodule of `PyEMMA`, and thus it also understands some of `PyEMMA`'s classes as input (like clustering objects or streaming transformers).
## Using the TICA object to visualize the most correlated input features
If the projected coordinates come from a TICA (or PCA) transformation, and the TICA object is available in memory
`molpx.visualize.traj` can make use of correlation information to display not only the projected coordinates (i.e the TICs, in this case), but also the "original" input features behind it

In [12]:
# Re-do the TICA computation to make sure we have a tica object in memory
feat = pyemma.coordinates.featurizer(top)
pairs = feat.pairs(range(feat.topology.n_atoms)[::2])
feat.add_distances(pairs)
src  = pyemma.coordinates.source(MD_trajfiles, features=feat)
tica = pyemma.coordinates.tica(src, lag=10, dim=3)
Y = tica.get_output()   

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

The method `molpx.visualize.correlations` tries to provide a visual representation of the projected coordinates by relating them to the input features, which carry more meaning, since they are (usually) familiar parameters such as atom distances, angles, contacts etc.

In [13]:
# Comment or uncomment the optinal parameters and see how the method reacts
# You can use a pre-instantiated the widget
#iwd = molpx.visualize._nglwidget_wrapper(MD_list[0][0])
# Or instantiate at the moment of calling visualize.correlations
iwd = None
corr, ngl_wdg = molpx.visualize.correlations(tica, 
                                          n_feats=3, 
                                          proj_idxs=[0,1,2], 
                                          geoms=MD_list[0][::100],
                                          #verbose=True,
                                          #proj_color_list=['red', 'blue', 'green'],
                                          widget=iwd
                                         )
ngl_wdg

A Jupyter Widget

Use the correlation-dictionary's modified `print` function to see what's inside in a human-friendly way

In [14]:
print(corr)

Correlation dictionary for 3 projections
 Corr[proj_0|feat] = 0.8
  DIST: PRO 9 CA 8 - TYR 23 CA 22
  feat nr. 112, atom idxs [ 8 22]
 Corr[proj_0|feat] = 0.7
  DIST: PRO 9 CA 8 - TYR 21 CA 20
  feat nr. 111, atom idxs [ 8 20]
 Corr[proj_0|feat] = 0.7
  DIST: PRO 9 CA 8 - LEU 29 CA 28
  feat nr. 115, atom idxs [ 8 28]

 Corr[proj_1|feat] = 0.7
  DIST: PRO 9 CA 8 - LYS 15 CA 14
  feat nr. 108, atom idxs [ 8 14]
 Corr[proj_1|feat] = 0.7
  DIST: LYS 15 CA 14 - TYR 23 CA 22
  feat nr. 178, atom idxs [14 22]
 Corr[proj_1|feat] = 0.6
  DIST: LYS 15 CA 14 - PHE 33 CA 32
  feat nr. 183, atom idxs [14 32]

 Corr[proj_2|feat] = 0.6
  DIST: THR 11 CA 10 - GLY 37 CA 36
  feat nr. 142, atom idxs [10 36]
 Corr[proj_2|feat] = -0.6
  DIST: PRO 13 CA 12 - GLN 31 CA 30
  feat nr. 161, atom idxs [12 30]
 Corr[proj_2|feat] = -0.6
  DIST: PRO 13 CA 12 - PHE 33 CA 32
  feat nr. 162, atom idxs [12 32]




Also, `molpx.visualize.traj` can help in visualizing these correlations by parsing along the tica object itself as `projection=tica`. In the next cell, can you spot the differences:
* In the nglwidget?
* In the trajectories?


In [15]:
# Reuse the visualize.traj method with the tica object as input
mpx_wdg_box = molpx.visualize.traj(MD_trajfiles,     
                                  top,                                                                                                                              
                                  Y,
                                  #Y_filenames,                                          
                                  plot_FES = True,                                        
                                  dt = dt*1e-6, tunits='ms',    
                                  #traj_selection = 0,
                                  #sharey_traj=False,
                                  #max_frames=100,
                                  proj_idxs=[0,1],                                           
                                  panel_height=1,
                                  projection=tica, ## this is what's new
                                  n_feats=2
                                 )
                          
mpx_wdg_box

A Jupyter Widget

A Jupyter Widget

30-05-18 17:43:37 pyemma.coordinates.clustering.kmeans.KmeansClustering[51] INFO     Cluster centers converged after 10 steps.


A Jupyter Widget

A Jupyter Widget

## Use a clustering object as input 
If the dataset has already been clustered, and it is **that** clustering that the user wants to explore, `molpx.generate.sample` can take this clustering object as an input instead of the 
the projected trajectories:

In [16]:
# Do "some" clustering
clkmeans = pyemma.coordinates.cluster_kmeans([iY[:,:2] for iY in Y], 5)

A Jupyter Widget

A Jupyter Widget

30-05-18 17:43:40 pyemma.coordinates.clustering.kmeans.KmeansClustering[60] INFO     Algorithm did not reach convergence criterion of 1e-05 in 10 iterations. Consider increasing max_iter.


In [17]:
data_sample, geoms = molpx.generate.sample(MD_trajfiles, top, clkmeans, 
                                     n_geom_samples=50, 
                                     #keep_all_samples=True # read the doc for this argument
                                    )

A Jupyter Widget

A Jupyter Widget

In [18]:
# Plot clusters
plt.figure(figsize=(4,4))
plt.plot(clkmeans.clustercenters[:,0], clkmeans.clustercenters[:,1],' ok')
# FES as background is optional (change the bool to False)
if True:
    plt.contourf(x[:-1], y[:-1], -np.log(h.T), alpha=.50)

# Link the clusters positions with the molecular structures
linked_ngl_wdg = molpx.visualize.sample(data_sample, 
                              geoms.superpose(geoms[0]),                             
                              plt.gca(), 
                              clear_lines=False,
                              #plot_path=True
                            )
linked_ngl_wdg

A Jupyter Widget

/home/mi/gph82/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


(NGLWidget(count=5), <matplotlib.widgets.AxesWidget at 0x7f56893edf98>)

## Visual representations for MSMs
Visually inspect the network behind an MSM by coarse graining it to an HMM

In [19]:
clkmeans = pyemma.coordinates.cluster_kmeans([iY[:,:2] for iY in Y], 100)
MSM = pyemma.msm.estimate_markov_model(clkmeans.dtrajs, 1)
MSMcg = MSM.coarse_grain(3)

A Jupyter Widget

A Jupyter Widget

30-05-18 17:43:43 pyemma.coordinates.clustering.kmeans.KmeansClustering[63] INFO     Cluster centers converged after 10 steps.


A Jupyter Widget

In [21]:
from imp import reload
from molpx import visualize
reload(visualize)

<module 'molpx.visualize' from '/home/mi/gph82/SOURCE_gph82/python/molPX/molpx/visualize.py'>

In [24]:
mpxbox = molpx.visualize.MSM(MSMcg, src, 
                             pos=clkmeans.clustercenters,
                              #sticky=True, 
                              sharpen=True, 
                              n_overlays=10,
                              figpadding=.5,
                              #proj_idxs =[1,0]
                             panelsize=6
             )
mpxbox

A Jupyter Widget

## TPT Reactive Pathway Representation
Until we add a method for doing this explicitly, you can do it in a few lines of code

In [ ]:
# Do an MSM with a realistic number of clustercenters
cl_many = pyemma.coordinates.cluster_regspace([iY[:,:2] for iY in Y], dmin=.25)
M = pyemma.msm.estimate_markov_model(cl_many.dtrajs, 20)
cl_many.n_clusters

In [ ]:
# Use this object to sample geometries
pos, geom = molpx.generate.sample(MD_trajfiles, top, cl_many)

In [ ]:
# Find the most representative microstate of each 
# and least populated macrostate
M.pcca(3)
dens_max_i = [distro.argmax() for distro in M.metastable_distributions]
A = np.argmax([M.stationary_distribution[iset].sum() for iset in M.metastable_sets])
B = np.argmin([M.stationary_distribution[iset].sum() for iset in M.metastable_sets])
print(cl_many.clustercenters[dens_max_i[A]],
      cl_many.clustercenters[dens_max_i[B]])

In [ ]:
# Create a TPT object with most_pop, least_pop as source, sink respectively
tpt = pyemma.msm.tpt(M, [dens_max_i[A]], [dens_max_i[B]])
paths, flux = tpt.pathways(fraction=.5)

In [ ]:
# Get a path with a decent number of intermediates
sample_path = paths[np.argmax([len(ipath) for ipath in paths])]

In [ ]:
plt.figure()
plt.contourf(x[:-1], y[:-1], -np.log(h.T), cmap="jet", alpha=.5, zorder=0)
linked_ngl_wdg, linked_ax_wdg = molpx.visualize.sample(cl_many.clustercenters[sample_path], 
                       geom[sample_path].superpose(geom[sample_path[0]]), plt.gca(), 
                       plot_path=True,
                      )
plt.scatter(*cl_many.clustercenters.T, alpha=.25)
linked_ngl_wdg